In [1]:
import tempfile
import matplotlib.pyplot as plt
import pydicom
from pydicom.data import get_testdata_file
import cv2
import os
from skimage import morphology
import math
import numpy as np
from scipy import ndimage
from tqdm import tqdm
import glob as glob
import SimpleITK as sitk
import sys
from pydicom.encaps import encapsulate
from pydicom.uid import JPEG2000
from pydicom.pixel_data_handlers.util import apply_modality_lut
from imagecodecs import jpeg_encode
import imagecodecs
import random
from pydicom.uid import ExplicitVRLittleEndian
from imagecodecs import jpeg_encode
import imagecodecs
import pylibjpeg
import timeit

# # CT

# In[ ]:



def sorted_files(folder):
    ins=[]
    ser=[]
    f= os.listdir(folder)
    for name in tqdm( f):
        itkimage = sitk.ReadImage(os.path.join(folder,name))
        temp= float(itkimage.GetMetaData('0020|0013')) # instance number 
        temp1= itkimage.GetMetaData('0020|000e') #series instance UID 
        ins.append(int(temp))
        ser.append(temp1)
    series=np.unique(ser)
    files=[[x for sr,_,x in sorted(zip(ser,ins,f)) if sr==s] for s in series]
    return files

def get_scan(path):
    f= sorted_files(path)
    series=[]
    files=[]
    for i in tqdm (range (len(f))):
        img=[]
        file=[]
        for filename in f[i]:
            itkimage = sitk.ReadImage(os.path.join(path,filename))
            numpyImage = sitk.GetArrayFromImage(itkimage)
            img.append(numpyImage[0,:,:])
            file.append(filename)
        series.append(img)
        files.append(file)
    return series,files

def CT_blurred_sag(image,dicom_img,rows,columns):
    imagee = image>-800
#     plt.imshow(imagee,'gray')
#     plt.title("image greater than -400")
#     plt.show()

    skull = image>1200
#     plt.imshow(skull,'gray')
#     plt.title("skull; image greater than 1000")
#     plt.show()

    kernel = np.ones((30, 30), np.uint8)
    imagee = cv2.morphologyEx(imagee.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)    
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,-1, 255,-1 )
#     plt.imshow(gray,'gray')
#     plt.title("contours")
#     plt.show()

    mid = int(image.shape[0]/2)

    gray = np.where((gray==255), 0, 255)
    gray[:,mid:] = 0
    gray1 = np.zeros_like(imagee)
    gray1 = cv2.drawContours(gray1, contours,-1, 255,10)
#     plt.imshow(gray1,'gray')
#     plt.title("contours2")
#     plt.show()
    gray1[:,mid:] = 0
    gray=np.where((gray1==255), gray1, gray)
    gray=np.where((gray==255),1,0)

#     plt.imshow(out,'gray')
#     plt.title("out")
#     plt.show()

    if np.min(image) == np.min(dicom_img) and np.max(image) == np.max(dicom_img):
#         print("equal")
        blurred_img = cv2.GaussianBlur(image, (101, 101), 400)
        out = np.where((gray==1), blurred_img, image)
        out = np.where(skull>0,image,out)
        out = out.astype(np.int16)
        print(np.min(out),np.max(out))
    else: 
#         print("not equal")
        blurred_img = cv2.GaussianBlur(dicom_img, (101, 101), 50)
        out = np.where((gray==1), blurred_img, dicom_img)
        out = np.where(skull>0,dicom_img,out)
        out = out.astype(np.int16)
        print(np.min(out),np.max(out))
        
#     for i in range(np.shape(out)[0]):
#         if not np.any(temp[i]):
#             out[i]=np.multiply(out[i],0)
#             out = out.astype(np.int16)
    
    return out
    
def CT_blurred_axial(image, dicom_img, rows, columns):
    imagee = image>-50
#     plt.imshow(imagee,'gray')
#     plt.title("image greater than -50")
#     plt.show()
    
    skull = image>500
#     plt.imshow(skull,'gray')
#     plt.title("skull; image greater than 500")
#     plt.show()
    
    kernel = np.ones((30, 30), np.uint8)
    imagee = cv2.morphologyEx(imagee.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)    
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,-1, 255,-1 )
#     plt.imshow(gray,'gray')
#     plt.title("contours")
#     plt.show()
    gray = np.where((gray==255), 0, 255)
    gray[300:,:] = 0
    gray1 = np.zeros_like(imagee)
    gray1 = cv2.drawContours(gray1, contours,-1, 255,7)
#     plt.imshow(gray1,'gray')
#     plt.title("contours2")
#     plt.show()
    gray1[300:,:] = 0
    gray=np.where((gray1==255), gray1, gray)
    gray=np.where((gray==255),1,0)
    
    if np.min(image) == np.min(dicom_img) and np.max(image) == np.max(dicom_img):
#         print("equal")
        blurred_img = cv2.GaussianBlur(image, (101, 101), 400)
        out = np.where((gray==1), blurred_img, image)
        out = np.where(skull>0,image,out)
        out = out.astype(np.int16)
        print(np.min(out),np.max(out))
        return out
    else: 
#         print("not equal")
        blurred_img = cv2.GaussianBlur(dicom_img, (101, 101), 50)
        out = np.where((gray==1), blurred_img, dicom_img)
        out = np.where(skull>0,dicom_img,out)
        out = out.astype(np.int16)
        print(np.min(out),np.max(out))
        return out
        
def CT_blurred_coronal(image, dicom_img, rows, columns):

    imagee = image>-400
#     plt.imshow(imagee,'gray')
#     plt.title("image greater than -400")
#     plt.show()

    skull = image>1000
#     plt.imshow(skull,'gray')
#     plt.title("skull; image greater than 1000")
#     plt.show()

    kernel = np.ones((30, 30), np.uint8)
    imagee = cv2.morphologyEx(imagee.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)    
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,-1, 255,-1 )
#     plt.imshow(gray,'gray')
#     plt.title("contours")
#     plt.show()

    mid = int(image.shape[0]/2)

    gray = np.where((gray==255), 0, 255)
    gray[:200,:] = 0
    gray1 = np.zeros_like(imagee)
    gray1 = cv2.drawContours(gray1, contours,-1, 255,10)
#     plt.imshow(gray1,'gray')
#     plt.title("contours2")
#     plt.show()
    gray1[:200,:] = 0
    gray=np.where((gray1==255), gray1, gray)
    gray=np.where((gray==255),1,0)

    if np.min(image) == np.min(dicom_img) and np.max(image) == np.max(dicom_img):
#         print("equal")
        blurred_img = cv2.GaussianBlur(image, (101, 101), 400)
        out = np.where((gray==1), blurred_img, image)
        out = np.where(skull>0,image,out)
        out = out.astype(np.int16)
        print(np.min(out),np.max(out))
    else: 
#         print("not equal")
        blurred_img = cv2.GaussianBlur(dicom_img, (101, 101), 50)
        out = np.where((gray==1), blurred_img, dicom_img)
        out = np.where(skull>0,dicom_img,out)
        out = out.astype(np.int16)
        print(np.min(out),np.max(out))
    
    return out

def predictions_ct(path, output):
    if not os.path.exists(output):
        os.makedirs(output)
    output_list= []
    classUID = []
#     start_files = timeit.default_timer()
    scan_series,filenames=get_scan(path)
#     stop_files = timeit.default_timer()
#     print("time for files: ", stop_files - start_files)
    
#     start_ai = timeit.default_timer()
    for j in tqdm(range(len(scan_series))):
        scan, names = scan_series[j],filenames[j]
        scan=np.asarray(scan)
        print(scan.shape[0])
        print("no.of files: ",len(names))
        dcmData = pydicom.dcmread(os.path.join(path,names[0]))
        
#         plt.imshow(dcmData.pixel_array,'gray')
#         plt.show()
        
        axial  =  np.array([1., 0., 0., 0., 1., 0.])
#         sagittal = np.array([-1, 0., 0., 0., 0., -1.])
        sagittal = np.array([0, 1., 0., 0., 0., -1.])
        coronal = np.array([1., 0., 0., 0., 0., -1.])
        if (0x0020,0x0037) in dcmData:
            a = (np.around(dcmData.ImageOrientationPatient, 0))
            print(a)
            if (a == axial).all():
#                 print("in axial")
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
#                     print("max and min of scan sitk ",np.min(img),np.max(img))
                    dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                    dicom_img = dcmData.pixel_array
#                     print("max and min of input dicom ",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
                    r= dcmData.Rows
                    c=dcmData.Columns
                    out= CT_blurred_axial(img,dicom_img,r,c)
                    out = out.astype(np.int16)
                    dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                    dcmData.PixelData = out.tobytes()
#                     print("max and min of output ",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
                    
#                     plt.imshow(dcmData.pixel_array,'gray')
#                     plt.title("axial blurred")
#                     plt.show()
                    
                    des_path = os.path.join(output, name)
                    dcmData.save_as(des_path) ##save file in anonym folder
                    classUID.append(str(dcmData.SOPClassUID))
                    output_list.append(des_path)
            elif (a==sagittal).all():
#                 print("in sagittal")
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
                    dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                    r= dcmData.Rows
                    c=dcmData.Columns
                    dicom_img = dcmData.pixel_array

                    out = CT_blurred_sag(img,dicom_img,r,c)
                        
                    out = out.astype(np.int16)
                    dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                    dcmData.PixelData = out.tobytes()
                    
#                     plt.imshow(dcmData.pixel_array,'gray')
#                     plt.title("sagittal blurred")
#                     plt.show()
                    
                    des_path = os.path.join(output, name)
                    dcmData.save_as(des_path) ##save file in anonym folder
                    classUID.append(str(dcmData.SOPClassUID))
                    output_list.append(des_path)
            elif (a==coronal).all():
#                 print("in coronal")
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
                    dcmData = pydicom.dcmread(os.path.join(path,name))
                    dicom_img = dcmData.pixel_array
#                     print("max and min of input dicom ",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
                    r= dcmData.Rows
                    c=dcmData.Columns
                    out= CT_blurred_coronal(img,dicom_img,r,c)
                    out = out.astype(np.int16)
                    dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                    dcmData.PixelData = out.tobytes()
#                     print("max and min of output ",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
                    
#                     plt.imshow(dcmData.pixel_array,'gray')
#                     plt.title("coronal blurred")
#                     plt.show()
                    
                    des_path = os.path.join(output, name)
                    dcmData.save_as(des_path) ##save file in anonym folder
                    classUID.append(str(dcmData.SOPClassUID))
                    output_list.append(des_path)
            else:
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
                    dcmData = pydicom.dcmread(os.path.join(path,name)) 
                    des_path = os.path.join(output, name)
                    dcmData.save_as(des_path) 
                    classUID.append(str(dcmData.SOPClassUID))
                    output_list.append(des_path)
        else:
            for i in range(scan.shape[0]):
                name = names[i]
                img = scan[i, :, :]
                dcmData = pydicom.dcmread(os.path.join(path,name)) 
                des_path = os.path.join(output, name)
                dcmData.save_as(des_path) 
                classUID.append(str(dcmData.SOPClassUID))
                output_list.append(des_path)
#     stop_ai = timeit.default_timer()
#     print("time for AI: ", stop_ai - start_ai)
#     mimeType = "application/dicom"
#     recommendation_string = {"finding": "finding","conclusion":"conclusion","recommendation":"recommendation"} 
#     return output_list, classUID, mimeType, recommendation_string
    return 1

In [2]:
def get_full_scan(folder_path):
    files_List = glob.glob(folder_path + '/**/*.dcm', recursive=True)
    itkimage = sitk.ReadImage(files_List[0])
    rows = int(itkimage.GetMetaData('0028|0010'))
    cols = int(itkimage.GetMetaData('0028|0011'))
    mn = 1000
    mx = 0
    for file in tqdm(files_List):
        itkimage = sitk.ReadImage(file)
        mn = np.min([mn, int(itkimage.GetMetaData('0020|0013'))])
        mx = np.max([mx, int(itkimage.GetMetaData('0020|0013'))])
    full_scan = np.ndarray(shape=(mx - mn + 1, rows, cols), dtype=float, order='F')
    new_list = np.ndarray(shape=(mx - mn + 1), dtype=object)
    for file in tqdm(files_List):
        img, n = dcm_image(file)
        n = int(n)
        full_scan[n - mn, :, :] = img[0, :, :]
        new_list[n-mn] = file
    return full_scan,new_list

def dcm_image(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
    ins = float(itkimage.GetMetaData('0020|0013'))
    return numpyImage, ins

In [3]:
def main(ipath, opath):
    predictions_ct(ipath, opath)

In [4]:
from skimage.metrics import structural_similarity

def signaltonoise(a, axis=0, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=ddof)
    return np.where(sd == 0, 0, m/sd)

In [5]:
ipath = r"F:/Face-Anonymization-main/CT/7019115_KimHwaKyoung/1.2.392.200036.9116.2.6.1.48.1211376315.1375921545.238520/Head_3_0_Sagittal_1"
opath = r"F:/Face-Anonymization-main/CT/7019115_KimHwaKyoung/1.2.392.200036.9116.2.6.1.48.1211376315.1375921545.238520/improved/Head_3_0_Sagittal_1"
ppath = r"F:/Face-Anonymization-main/CT/7019115_KimHwaKyoung/1.2.392.200036.9116.2.6.1.48.1211376315.1375921545.238520/output/Head_3_0_Sagittal_1"
# main(ipath, opath)

In [6]:
def get_imgs_score(ipath, ppath, opath):
    iscan, n = get_full_scan(ipath)
    oscan, n = get_full_scan(opath)
    pscan, n = get_full_scan(ppath) 
    
    i = int(iscan.shape[0]/2)
    print(i)
    plt.imshow(iscan[i],'gray')
    plt.show()
    plt.imshow(pscan[i],'gray')
    plt.show()
    plt.imshow(oscan[i],'gray')
    plt.show()   
    
    snri = signaltonoise(iscan, axis=None, ddof=0)
    snro = signaltonoise(oscan, axis=None, ddof=0)
    score,diff = structural_similarity(iscan, oscan, full=True)
    print(snri, snro, score) 

In [7]:
# get_imgs_score(ipath, ppath, opath)